### **AgroHack - soybeans yield prediction**

---

#### **Постановка задачи**
Наша цель — спрогнозировать урожайность сои, используя ее генотипические данные и данные о погодных условиях в регионе высаживания. Для этого мы разработали подход, который позволяет перевести биологические последовательности (нуклеотидный набор, представляющий собой их последовательность с учетом мутаций) в числовые представления, а затем применить машинное обучение для создания предсказательной модели. Этот ноутбук посвящен работе с био фичами и последующим обучением модели вместе с погодой.

---

#### **Методы построения фичей**
Для перевода биологических данных в цифровую форму мы использовали несколько ключевых этапов:

1. **Сбор и обработка генотипических данных**:
   - Данные представлены в формате VCF, содержащем информацию о референсных и альтернативных аллелях для каждой позиции генома.
   - Мы упорядочили данные по хромосоме и позиции, обработали пропуски, чтобы сохранить целостность последовательностей.

2. **Цифровизация последовательностей через `k-mers`**:
   - Длинные нуклеотидные последовательности были разбиты на подстроки фиксированной длины \( k=3 \), так как триплет представляет собой одну аминокислоту.

3. **Эмбеддинги с использованием Word2Vec**:
   - `k-mers` были преобразованы в числовые векторы с использованием модели Word2Vec, которая извлекает контекстные зависимости между ними.
   - Итоговое представление каждой последовательности — это усредненный вектор всех её `k-mers`. Это компактное числовое представление генотипов, готовое для использования в моделях машинного обучения.

---

#### **Baseline**
1. **Работа по его построению**:
   - Каждый генотип представляет собой набор из ~40 тысяч фичей, обработка классическими моделями получилась вычислительно долгой и совершенно неоптимизированной
   - Обработка данных о количестве мутаций для снижения числа фичей и как способ предсказания не дал положительных результатов - 98% дисперсии такая модель обьяснить не может
   - Обработка данных о погоде (соседний ноутбук)
   - Оценка качества на простых моделях (RandomForestRegressor)

---
#### **Методы анализа**
1. **Модели машинного обучения**:
   - Мы использовали CatBoostRegressor для построения модели, которая предсказывает урожайность на основе созданных эмбеддингов и погодных факторов.
   - CatBoost, как градиентный бустинг, показал себя наиболее эффективно благодаря встроенной обработке числовых данных и оптимизации.
   - Также, мы использовали библиотеку optuna для подбора наилучших гиперпараметров модели

2. **Метрики оценки**:
   - Для оценки точности модели мы применили метрики MSE (среднеквадратическая ошибка) и \( R^2 \) (коэффициент детерминации).
   - Эти метрики помогли нам определить, насколько модель отражает реальные данные.

---

#### **Итог**
Мы продемонстрировали, как генетические данные могут быть цифровизированы и использованы для предсказания продуктивности растений. Этот подход:

- Основан на биологических закономерностях, таких как структура `k-mers` в ДНК.
- Применяет современные методы эмбеддингов для создания информативных фичей.
- Использует мощные модели машинного обучения для получения точных прогнозов.
- Качество модели: CatBoost Mean Squared Error (RMSE): **15.2**; CatBoost R^2 Score: **0.163**
- Мы получили решение, с помощью которого в дальнейшем важность отдельных эмбеддингов можно интерпретировать как важность мутации конкретных генов и, отталкиваясь от этого, планировать селекцию и экспериментировать с получившимися генотипами
---


In [3]:
!pip install catboost gensim optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 1.4 MB/s eta 0:00:0000:0100:01m


In [4]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from catboost import CatBoostRegressor
import optuna

In [5]:
def wide_to_long(phenotypes_df:pd.DataFrame(),  value_vars:list, id_vars='sample') -> pd.DataFrame():
    """
    Преобразует широкую таблицу с таргетами в длинную.
    
    :param df: Исходный DataFrame в широком формате.
    :param id_vars: sample, которые остаются идентификаторами.
    :param value_vars: Года, которые нужно преобразовать в длинный формат.
    :return: DataFrame в длинном формате.
    """
    if value_vars is None:
        value_vars = ['2015', '2016', '2017', '2019', '2020', '2021', '2022', '2023']
        
    target_df = (pd.melt(phenotypes_df, id_vars=['sample'], value_vars=value_vars)
             .dropna()
             .rename(columns={'variable':'year',
                              'value':'yield'}
                    )
            )
    return target_df

In [6]:
# извлекаем аллели
def extract_allele(gt, ref, alt):
    """
    Достаем аллель по типу мутации
    
    :param gt: мутация
    :param ref: референсный аллель.
    :param alt: альтернатиыный аллель
    :return: нуклеотидная последовательность
    """
    if gt == "./.":
        return "" 
    alleles = gt.split(":")[0].split("/") 
    seq = "".join([ref if a == "0" else alt if a != "0" else "" for a in alleles])
    return seq

# разбиение последовательности на k-mers
def generate_kmers(sequence, k=3):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

In [7]:
def read_vcf(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if not line.startswith('##')]  # Пропускаем метаданные
    headers = lines[0].split('\t')  # Заголовки колонок
    data = [line.split('\t') for line in lines[1:]]  # Данные
    vcf_df = pd.DataFrame(data, columns=headers)
    return vcf_df

In [8]:
# чтение файлов
genotypes = read_vcf('genotypes.vcf')
raw_phenotypes = pd.read_csv('phenotypes.tsv', sep='\t')

In [9]:
# извлечение образцов и построение последовательностей
sample_columns = genotypes.columns[9:]  # Колонки с образцами
sample_sequences = {sample: [] for sample in sample_columns}

genotypes["POS"] = pd.to_numeric(genotypes["POS"], errors="coerce")
genotypes.sort_values(by=["#CHROM", "POS"], inplace=True)  # сортировка по хромосоме и позиции

for _, row in genotypes.iterrows():
    ref = row["REF"]
    alt = row["ALT"]
    for sample in sample_columns:
        gt = row[sample]
        allele_seq = extract_allele(gt, ref, alt)
        sample_sequences[sample].append(allele_seq)

# объединяем последовательности в строки
sample_sequences = {sample: "".join(seq) for sample, seq in sample_sequences.items()}


In [10]:

# генерация k-mers и обучение Word2Vec
k = 3  # Размер k-mers
kmers_samples = {sample: generate_kmers(seq, k=k) for sample, seq in sample_sequences.items()}
sentences = list(kmers_samples.values())

# обучение модели Word2Vec
w2v_model = Word2Vec(sentences, vector_size=200, window=6, sg=1, epochs=30)  # skip-gram


In [11]:
# генерация эмбеддингов (среднее по k-mers)
embeddings = {
    sample: sum(w2v_model.wv[mer] for mer in kmers) / len(kmers) if kmers else None
    for sample, kmers in kmers_samples.items()
}

# Подготовка данных: объединяем эмбеддинги с target
embedding_df = pd.DataFrame.from_dict(embeddings, orient='index', dtype=float)
embedding_df.index.name = "sample"
embedding_df.reset_index(inplace=True)

In [12]:
target_data = wide_to_long(raw_phenotypes, ['2015', '2016', '2017', '2019', '2020', '2021', '2022', '2023'])

In [13]:
weather_features = pd.read_csv('res_meteo.csv') # подключаем фичи с погодой
weather_features.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [14]:
# Объединяем с данными по урожайности
merged_data = pd.merge(target_data, embedding_df, on="sample", how="inner")

In [19]:
merged_data.year = merged_data.year.astype(int)

In [20]:
# Объединяем с данными по погоде
merged_data = pd.merge(merged_data, weather_features, on="year", how="inner")

In [21]:
# Разделение на признаки и целевую переменную
X = merged_data.drop(columns=["sample", "year", "yield"])
y = merged_data["yield"]

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# проводим отбор гиперпараметров
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

[I 2024-11-24 11:28:20,722] A new study created in memory with name: no-name-25f77a16-8b05-416e-aa36-0a52522d476b
[I 2024-11-24 11:28:23,642] Trial 0 finished with value: 15.03968543977089 and parameters: {'learning_rate': 0.0024339015645837687, 'depth': 5, 'subsample': 0.6952686998445793, 'colsample_bylevel': 0.944742534372284, 'min_data_in_leaf': 35}. Best is trial 0 with value: 15.03968543977089.
[I 2024-11-24 11:28:24,223] Trial 1 finished with value: 16.09352367083085 and parameters: {'learning_rate': 0.0996272375146593, 'depth': 2, 'subsample': 0.6297373378410382, 'colsample_bylevel': 0.30147257225587065, 'min_data_in_leaf': 54}. Best is trial 0 with value: 15.03968543977089.


In [ ]:
# для упрощения работы с ноутбуком выведу параметры отдельно
best_params = {'learning_rate': 0.0024339015645837687, 'depth': 5, 'subsample': 0.6952686998445793, 'colsample_bylevel': 0.944742534372284, 'min_data_in_leaf': 35}

In [ ]:
# Вес обратно пропорционален частоте - попробовали использовать взвешивание таргетов, но это не дало значимого прироста
# target_counts = y_train.value_counts(normalize=True)
# weights = y_train.map(lambda x: 1 / target_counts[x])  

# обучение модели CatBoost
catboost_model = CatBoostRegressor(
    **best_params, 
    verbose=100
)

catboost_model.fit(X_train,
                   y_train,
                   # sample_weight=weights,
                   eval_set=(X_test, y_test),
                   verbose=100)

# предсказания и оценка
y_pred_cb = catboost_model.predict(X_test)
mse_cb = mean_squared_error(y_test, y_pred_cb)
r2_cb = r2_score(y_test, y_pred_cb)

print("CatBoost Mean Squared Error (MSE):", mse_cb)
print("CatBoost R^2 Score:", r2_cb)


In [ ]:
from catboost import Pool
import matplotlib.pyplot as plt
import seaborn as sns

# посмотрим на FI
def plot_feature_importance(importance, names, model_type):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,))
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

plot_feature_importance(catboost_model.get_feature_importance(),X_train.columns,'CATBOOST')